Grab fablib manager

In [9]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
import json
import shlex
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,sjzhu_0000105651
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Find a site

In [11]:
# find a site with appropriate cores
cores_per_node = 16
ram_per_node = 32
disk_per_node = 20 # we will modify this later to give fileservers more
site1 = fablib.get_random_site(
    filter_function=lambda x: x["cores_available"] > cores_per_node * 3 and x["ram_available"] > ram_per_node * 3 and x["disk_available"] > disk_per_node * 3 and x["nic_connectx_6_available"] > 2
)
hosts = [f"{site1.lower()}-c1", f"{site1.lower()}-r1", f"{site1.lower()}-p1"]
fablib.show_site(site1, 'json')

{
    "name": "AMST",
    "state": "Active",
    "address": "Science Park 904, 1098 XH Amsterdam",
    "location": [
        52.3545559,
        4.9558617
    ],
    "hosts": 3,
    "cpus": 6,
    "cores_available": 361,
    "cores_capacity": 384,
    "cores_allocated": 23,
    "ram_available": 1012,
    "ram_capacity": 1506,
    "ram_allocated": 494,
    "disk_available": 63298,
    "disk_capacity": 64068,
    "disk_allocated": 770,
    "nic_basic_available": 361,
    "nic_basic_capacity": 381,
    "nic_basic_allocated": 20,
    "nic_connectx_6_available": 3,
    "nic_connectx_6_capacity": 3,
    "nic_connectx_6_allocated": 0,
    "nic_connectx_5_available": 2,
    "nic_connectx_5_capacity": 2,
    "nic_connectx_5_allocated": 0,
    "nvme_available": 10,
    "nvme_capacity": 10,
    "nvme_allocated": 0,
    "tesla_t4_available": 0,
    "tesla_t4_capacity": 0,
    "tesla_t4_allocated": 0,
    "rtx6000_available": 0,
    "rtx6000_capacity": 0,
    "rtx6000_allocated": 0,
    "a30_availa

'{\n    "name": "AMST",\n    "state": "Active",\n    "address": "Science Park 904, 1098 XH Amsterdam",\n    "location": [\n        52.3545559,\n        4.9558617\n    ],\n    "hosts": 3,\n    "cpus": 6,\n    "cores_available": 361,\n    "cores_capacity": 384,\n    "cores_allocated": 23,\n    "ram_available": 1012,\n    "ram_capacity": 1506,\n    "ram_allocated": 494,\n    "disk_available": 63298,\n    "disk_capacity": 64068,\n    "disk_allocated": 770,\n    "nic_basic_available": 361,\n    "nic_basic_capacity": 381,\n    "nic_basic_allocated": 20,\n    "nic_connectx_6_available": 3,\n    "nic_connectx_6_capacity": 3,\n    "nic_connectx_6_allocated": 0,\n    "nic_connectx_5_available": 2,\n    "nic_connectx_5_capacity": 2,\n    "nic_connectx_5_allocated": 0,\n    "nvme_available": 10,\n    "nvme_capacity": 10,\n    "nvme_allocated": 0,\n    "tesla_t4_available": 0,\n    "tesla_t4_capacity": 0,\n    "tesla_t4_allocated": 0,\n    "rtx6000_available": 0,\n    "rtx6000_capacity": 0,\n    "r

In [12]:
slice_name = 'ndn-poc-sjzhu2'

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

In [4]:
# Define a subnet to use for the L2 Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]

In [13]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

for h in hosts:

    node = slice.add_node(name=f"{h}-a", cores=cores_per_node, ram=ram_per_node, disk=disk_per_node, site=site1, image='default_ubuntu_22')
    node.add_fabnet(nic_type='NIC_ConnectX_6')
    node.add_post_boot_upload_directory("scripts")
    node.add_post_boot_execute("./scripts/install_dependencies.sh")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")

#Submit Slice Request
slice.submit();


Retry: 10, Time: 974 sec


ID,055938fe-72b4-4c30-a214-e9366085b616
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-21 00:19:18 +0000
Lease Start (UTC),2023-11-20 00:19:18 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
115d7476-80c7-48a2-8dff-e3622defeff7,amst-c1-a,16,32,100,default_ubuntu_22,qcow2,amst-w2.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fed6:327c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fed6:327c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
9ec275cf-422c-4e3f-8b55-e778a01ab659,amst-p1-a,16,32,100,default_ubuntu_22,qcow2,amst-w2.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe36:9c43,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe36:9c43,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2f619a80-0f7a-428d-88a1-33427bac918a,amst-r1-a,16,32,100,default_ubuntu_22,qcow2,amst-w2.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:feb3:6b65,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:feb3:6b65,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
64c1915f-f4ec-463e-be78-c9e694b1d473,FABNET_IPv4_AMST,L3,FABNetv4,AMST,10.145.2.0/24,10.145.2.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
amst-c1-a-FABNET_IPv4_AMST_nic-p1,p1,amst-c1-a,FABNET_IPv4_AMST,100,auto,,10:70:FD:E5:C2:18,enp7s0np0,enp7s0np0,10.145.2.3,6
amst-c1-a-FABNET_IPv4_AMST_nic-p2,p2,amst-c1-a,None,100,config,,10:70:FD:E5:C2:19,enp8s0np0,enp8s0np0,None,6
amst-r1-a-FABNET_IPv4_AMST_nic-p2,p2,amst-r1-a,None,100,config,,10:70:FD:E5:CD:D1,enp8s0np0,enp8s0np0,None,2
amst-r1-a-FABNET_IPv4_AMST_nic-p1,p1,amst-r1-a,FABNET_IPv4_AMST,100,auto,,10:70:FD:E5:CD:D0,enp7s0np0,enp7s0np0,10.145.2.4,2
amst-p1-a-FABNET_IPv4_AMST_nic-p1,p1,amst-p1-a,FABNET_IPv4_AMST,100,auto,,10:70:FD:E5:E7:08,enp7s0np0,enp7s0np0,10.145.2.2,1
amst-p1-a-FABNET_IPv4_AMST_nic-p2,p2,amst-p1-a,None,100,config,,10:70:FD:E5:E7:09,enp8s0np0,enp8s0np0,None,1



Time to print interfaces 980 seconds


Here are all of the settings for the various NDN services that need to be booted up.

In [24]:
FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}
FS_ACTIVATE = {
    'eal': {
        'memPerNuma': {'0': 4*1024},
        'filePrefix': 'producer',
    },
    'mempool': {
        'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**16-1},
        'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
    },
    'face': {
        'scheme': 'memif',
        'socketName': MEMIF_ARGS['socketName'],
        'id': MEMIF_ARGS['id'],
        'dataroom': 9000,
        'role': 'client',
    },
    'fileServer': {
        'mounts': [
            {'prefix': "/producer/usr-local-share", 'path': "/usr/local/share"}
        ],
        'segmentLen': 6 * 1024,
    },
}

adj_list = {
    hosts[0]: [hosts[1]],
    hosts[1]: [hosts[0], hosts[2]],
    hosts[2]: [hosts[1]],
}

forwarder_hashes = {}
face_hashes = {}

In [25]:
# add eth ports to each node
for h in hosts:
    node = slice.get_node(f"{h}-a")
    output = node.execute(f'''
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif enp7s0np0 --xdp --mtu 1500
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

# add interface between nodes in adj_list
for h in adj_list:
    local_mac = forwarder_hashes[h]['macAddr']
    node = slice.get_node(f"{h}-a")
    for remote in adj_list[h]:
        remote_mac = forwarder_hashes[remote]['macAddr']
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
        face_hashes[f"{h}:{remote}"] = json.loads(output[0])

true
{"id":"HGJBENMTGARTM","isDown":false,"macAddr":"10:70:fd:e5:c2:18","mtu":3000,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"V38TL1SJ24NCM","isDown":false,"macAddr":"10:70:fd:e5:cd:d0","mtu":3000,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"MO0FJC1PT8V1S","isDown":false,"macAddr":"10:70:fd:e5:e7:08","mtu":3000,"name":"net_af_xdp_3","numaSocket":null}
{"id":"HSPROVS2PIUD6H0"}
{"id":"VF2D39MCAGJSGIQ8"}
{"id":"VF2D39MCACIC4IIA"}
{"id":"MKAV54B6LOQHLGK8"}


In [26]:
print(forwarder_hashes)
print(face_hashes)
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
print(f"{hosts[0]}:{hosts[1]}")

{'amst-c1': {'id': 'HGJBENMTGARTM', 'isDown': False, 'macAddr': '10:70:fd:e5:c2:18', 'mtu': 3000, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'amst-r1': {'id': 'V38TL1SJ24NCM', 'isDown': False, 'macAddr': '10:70:fd:e5:cd:d0', 'mtu': 3000, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'amst-p1': {'id': 'MO0FJC1PT8V1S', 'isDown': False, 'macAddr': '10:70:fd:e5:e7:08', 'mtu': 3000, 'name': 'net_af_xdp_3', 'numaSocket': None}}
{'amst-c1:amst-r1': {'id': 'HSPROVS2PIUD6H0'}, 'amst-r1:amst-c1': {'id': 'VF2D39MCAGJSGIQ8'}, 'amst-r1:amst-p1': {'id': 'VF2D39MCACIC4IIA'}, 'amst-p1:amst-r1': {'id': 'MKAV54B6LOQHLGK8'}}
amst-c1:amst-r1


In [27]:
# add fileserver configuration to producer
producer_name = hosts[2] # hardcode this for now
producer_node = slice.get_node(f"{producer_name}-a")
output = producer_node.execute(f'''
    MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
    ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh $(echo $MEMIF_FACE | jq -r .id)
    ''')
# start fileserver instance
output = producer_node.execute(f'''
    sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
    echo {shlex.quote(json.dumps(FS_ACTIVATE))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
    ''')

{"id":"MKEV7C9IT1R5FJTJPTI6KAFH6SNA0T0"}
true


In [28]:
# add FIB entries to point to producer
c_node = slice.get_node(f"{hosts[0]}-a")
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
output = c_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {c_to_r['id']}")

r_node = slice.get_node(f"{hosts[1]}-a")
r_to_p = face_hashes[f"{hosts[1]}:{hosts[2]}"]
r_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {r_to_p['id']}")

{"id":"HSTRQNUMG3VP4ID4CKASJOQE7UGSQ78"}
{"id":"VF6D11KO2DJ84GRHV83ODVF1GGB3VH0"}


('{"id":"VF6D11KO2DJ84GRHV83ODVF1GGB3VH0"}\n', '')

In [21]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    slice = fablib.get_slice(slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

In [22]:
slice.update()

In [23]:
slice = fablib.get_slice(slice_name)
slice.show()

ID,055938fe-72b4-4c30-a214-e9366085b616
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-24 00:43:37 +0000
Lease Start (UTC),2023-11-20 00:19:18 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,055938fe-72b4-4c30-a214-e9366085b616
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-24 00:43:37 +0000
Lease Start (UTC),2023-11-20 00:19:18 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


TODO:
- script forwarder setup
  - use example activation parameters, excluding the CPU cores
- script forwarders to connect to each other
  - will need mac address from each and then create ether-face between pairs
- script fileserver setup
  - use example activation parameters, possibly editing the mountpoints
- use fileserver docs to setup FIB to fileserver using /producerN name
- create FIB entries for fileservers in each forwarder

In [27]:
slice = fablib.get_slice(slice_name)
node = slice.get_node("star-p1-a")
node.upload_file("BigBuckBunny-ratelimiter.mp4", "BBB.mp4")

<SFTPAttributes: [ size=67295074 uid=1000 gid=1000 mode=0o100664 atime=1699830987 mtime=1699830992 ]>

In [7]:
slice.delete()